In [6]:
# !pip install bitsandbytes
# !pip uninstall fitz -y
# !pip install PyMuPDF
# !pip install -q "transformers[torch]" pymupdf pillow

In [ ]:
# !pip install ipywidgets

In [4]:
from huggingface_hub import notebook_login

# Log in to the Hugging Face Hub
notebook_login()
if notebook_login == True:
  print("Logged in to the Hugging Face Hub.")
else:
  print("Failed to log in to the Hugging Face Hub.")

Failed to log in to the Hugging Face Hub.


In [7]:
from transformers import pipeline

pipe = pipeline(
    "image-text-to-text",
    model="unsloth/gemma-3-4b-it-bnb-4bit",
    model_kwargs={"torch_dtype": "auto"},
    device_map="auto"
)


c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--unsloth--gemma-3-4b-it-bnb-4bit. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
The installed version of bitsandbytes was compiled without GPU support. 8-bit optim

In [ ]:
import fitz  # PyMuPDF
import base64
from PIL import Image
from io import BytesIO

def pdf_to_images_base64(pdf_path):
    doc = fitz.open(pdf_path)
    images_base64 = []
    for page in doc:
        pix = page.get_pixmap(dpi=200)
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        buffered = BytesIO()
        img.save(buffered, format="JPEG")
        encoded = base64.b64encode(buffered.getvalue()).decode("utf-8")
        images_base64.append(encoded)
    return images_base64

def run_pdf_vision_prompt(pipe, pdf_path, question, system_prompt="You are a helpful assistant.", max_tokens=200):
    base64_images = pdf_to_images_base64(pdf_path)
    responses = []

    for i, image_b64 in enumerate(base64_images):
        messages = [
            {
                "role": "system",
                "content": [{"type": "text", "text": system_prompt}]
            },
            {
                "role": "user",
                "content": [
                    {"type": "image", "image": f"data:image/jpeg;base64,{image_b64}"},
                    {"type": "text", "text": question}
                ]
            }
        ]
        output = pipe(text=messages, max_new_tokens=max_tokens)
        response_text = output[0]["generated_text"][-1]["content"]
        responses.append((f"Page {i+1}", response_text))

    return responses


In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload your PDF

pdf_path = next(iter(uploaded))
question = "tell me how many people are there in the image their are some at back dont miss them  "

responses = run_pdf_vision_prompt(pipe, pdf_path, question)

for page, reply in responses:
    print(f"\n{page}:\n{reply}\n")


Saving image (1)1 half.jpg to image (1)1 half.jpg

Page 1:
Okay, let's count the people in the image!

Based on what I can see, there are approximately **14-15 people** visible in the image. 

I've included everyone, including those at the back of the bus. 

Let me know if you'd like me to focus on counting specific groups or individuals!



In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload your PDF

pdf_path = next(iter(uploaded))
question = "tell me how many people are there in the image their are some at back dont miss them you sometimes can only see their heads or hairs dive htem into category like fully visible, partically visible, only head or hairs visible etc"

responses = run_pdf_vision_prompt(pipe, pdf_path, question)

for page, reply in responses:
    print(f"\n{page}:\n{reply}\n")


Saving image (1)1 half.jpg to image (1)1 half (1).jpg

Page 1:
Okay, let's analyze the image and estimate the number of people and their visibility levels:

**Overall Count:**

Based on the image, I can identify approximately **14-16** people in the bus. 

**Detailed Breakdown by Visibility:**

*   **Fully Visible:** 4 people (clearly seen from head to torso)
*   **Partially Visible:** 6 people (some visible from the torso or waist up)
*   **Only Head/Hairs Visible:** 4-6 people (only heads and/or hair are visible - likely obscured by others or seated further back).
    *   I see some individuals sitting further back in the bus, with just their heads or hair visible. 


**Important Note:** It's difficult to be entirely precise due to the perspective and some partial obstructions in the image. 

Do you want me to focus on any specific area or group of people in the image, or perhaps try



In [ ]:
from google.colab import files
import time
uploaded = files.upload()  # Upload your PDF

pdf_path = next(iter(uploaded))
question = "Extract PII (Personally identifiable information) from given file"

responses = run_pdf_vision_prompt(pipe, pdf_path, question)

for page, reply in responses:
    print(f"\n{page}:\n{reply}\n")


Saving ilide.info-137-cosv-council-of-orthodox-synagogues-of-victoria-ltd-current-historical-compa-pr_9050a586eb2092a568687035a1980e5e.pdf to ilide.info-137-cosv-council-of-orthodox-synagogues-of-victoria-ltd-current-historical-compa-pr_9050a586eb2092a568687035a1980e5e.pdf

Page 1:
Okay, I've analyzed the provided image of the ASIC Extract.

**Here’s the Personally Identifiable Information (PII) extracted:**

*   **Name:** COUNCIL OF ORTHODOX SYNAGOGUES OF VICTORIA LTD.
*   **ACN:** 110 325 463

**Important Notes:**

*   This extract primarily provides company-level information, not individual personal details.

**Disclaimer:** I am an AI assistant and cannot provide legal advice. This information is based solely on the image provided.


Page 2:
Here's an extraction of Personally Identifiable Information (PII) from the provided file, based on the information available:

**Organization Details:**

*   **Name:** COUNCIL OF ORTHODOX SYNAGOGUES OF VICTORIA LTD.
*   **ACN:** 110 325 463
*  

In [ ]:
from google.colab import files
import time

start_time = time.time()  # Start timing

uploaded = files.upload()  # Upload your PDF
pdf_path = next(iter(uploaded))
question = "Extract PII (Personally identifiable information) from given file"

responses = run_pdf_vision_prompt(pipe, pdf_path, question)

for page, reply in responses:
    print(f"\n{page}:\n{reply}\n")

end_time = time.time()  # End timing

print(f"\nExecution Time: {end_time - start_time:.2f} seconds")


Saving ilide.info-050-flinders-trustees-limited-current-historical-company-extract-pr_4068e05353b00a72d5c022538da00c8c.pdf to ilide.info-050-flinders-trustees-limited-current-historical-company-extract-pr_4068e05353b00a72d5c022538da00c8c (1).pdf

Page 1:
Here's the personally identifiable information (PII) extracted from the provided file:

*   **Name:** FLINDERS TRUSTEES LIMITED
*   **ACN:** 081 567 204

**Note:**  This document is a public ASIC extract and doesn't contain personal details of individuals connected to the company.


Page 2:
Okay, let's carefully extract Personally Identifiable Information (PII) from the provided document. Here's a breakdown of the PII present, categorized for clarity:

**1. Company Details (Directly Identifying):**

*   **Name:** FLINDERS TRUSTEES LIMITED (This is a key identifier)
*   **ACN:** 081567204 (Unique Australian Company Number - very identifying)
*   **ABN:** 11081567204 (Unique Australian Business Number - extremely identifying)

**2. Conta